<a href="https://colab.research.google.com/github/BianPaulParaguya/Options-Screener/blob/main/OptionsPickerUnder1Week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install imblearn

In [ ]:
  import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import time
from IPython.display import display, clear_output

def get_stock_data(ticker, period="1y", interval="1d"):
    stock = yf.Ticker(ticker)
    hist = stock.history(period=period, interval=interval)
    return hist

def calculate_macd(data):
    short_ema = data['Close'].ewm(span=12, adjust=False).mean()
    long_ema = data['Close'].ewm(span=26, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=9, adjust=False).mean()
    return macd, signal

def calculate_rsi(data, period=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_bollinger_bands(data, window=20):
    sma = data['Close'].rolling(window=window).mean()
    std = data['Close'].rolling(window=window).std()
    upper_band = sma + (std * 2)
    lower_band = sma - (std * 2)
    return upper_band, lower_band

def get_option_chain(ticker):
    stock = yf.Ticker(ticker)
    options = stock.options
    if options:
        exp = options[0]  # Nearest expiration date
        option_chain = stock.option_chain(exp)
        return option_chain, exp
    return None, None

def get_implied_volatility(option_chain):
    calls = option_chain.calls
    puts = option_chain.puts
    avg_iv = (calls['impliedVolatility'].mean() + puts['impliedVolatility'].mean()) / 2
    return avg_iv

def select_option(option_chain, suggestion, price):
    if suggestion == "Call":
        options = option_chain.calls
        options = options[options['strike'] <= price]  # ITM or ATM
    else:
        options = option_chain.puts
        options = options[options['strike'] >= price]  # ITM or ATM

    if options.empty:
        return None

    # Calculate breakeven points
    options['breakeven'] = options['strike'] + options['lastPrice']
    options['percent_to_breakeven'] = ((options['breakeven'] - price) / price) * 100

    # Calculate potential profit (assuming stock moves to strike price)
    options['potential_profit'] = abs(options['strike'] - price) - options['lastPrice']

    # Select option with the highest potential profit
    selected_option = options.loc[options['potential_profit'].idxmax()]

    return selected_option

def analyze_stock(ticker):
    data = get_stock_data(ticker)
    macd, signal = calculate_macd(data)
    rsi = calculate_rsi(data)
    upper_band, lower_band = calculate_bollinger_bands(data)

    option_chain, expiration = get_option_chain(ticker)
    if option_chain is None:
        return {"Error": "No options data available for this ticker."}

    iv = get_implied_volatility(option_chain)

    latest_macd = macd.iloc[-1]
    latest_signal = signal.iloc[-1]
    latest_rsi = rsi.iloc[-1]
    latest_upper_band = upper_band.iloc[-1]
    latest_lower_band = lower_band.iloc[-1]
    latest_close = data['Close'].iloc[-1]

    suggestion = "Hold"
    action = "None"
    order_type = "None"
    time_in_force = "None"

    if latest_macd > latest_signal and latest_rsi < 70 and latest_close < latest_upper_band:
        suggestion = "Call"
        action = "Buy"
        order_type = "Limit"
        time_in_force = "Day"
    elif latest_macd < latest_signal and latest_rsi > 30 and latest_close > latest_lower_band:
        suggestion = "Put"
        action = "Buy"
        order_type = "Limit"
        time_in_force = "Day"

    if iv > 0.5:
        suggestion += " (high volatility - consider risks)"
        time_in_force = "GTC"

    price = data['Close'].iloc[-1]
    selected_option = select_option(option_chain, suggestion.split()[0], price)

    if selected_option is None:
        return {"Error": "No suitable options found."}

    exp_date = datetime.strptime(expiration, '%Y-%m-%d')
    today = datetime.today()
    days_to_exp = (exp_date - today).days
    dte_str = f"{days_to_exp}dte" if days_to_exp > 0 else "0dte"

    return {
        "MACD": latest_macd,
        "Signal": latest_signal,
        "RSI": latest_rsi,
        "Upper Band": latest_upper_band,
        "Lower Band": latest_lower_band,
        "Implied Volatility": iv,
        "Suggestion": suggestion,
        "Action": action,
        "Order Type": order_type,
        "Time in Force": time_in_force,
        "Option Expiration": expiration,
        "Option Strike Price": selected_option['strike'],
        "Option Premium": selected_option['lastPrice'],
        "Option Breakeven": selected_option['breakeven'],
        "Percent to Breakeven": selected_option['percent_to_breakeven'],
        "Potential Profit": selected_option['potential_profit'],
        "Summary": f"${ticker} {selected_option['strike']} {suggestion.split()[0].upper()} ({dte_str}) @{selected_option['lastPrice']:.2f}"
    }

def track_stock(ticker, interval=60):
    while True:
        data = []
        analysis = analyze_stock(ticker)
        if "Error" in analysis:
            data.append([ticker, analysis["Error"], "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", ""])
        else:
            data.append([
                ticker,
                analysis['MACD'],
                analysis['Signal'],
                analysis['RSI'],
                analysis['Upper Band'],
                analysis['Lower Band'],
                analysis['Implied Volatility'],
                analysis['Suggestion'],
                analysis['Action'],
                analysis['Order Type'],
                analysis['Time in Force'],
                analysis['Option Expiration'],
                analysis['Option Strike Price'],
                analysis['Option Premium'],
                analysis['Option Breakeven'],
                analysis['Percent to Breakeven'],
                analysis['Potential Profit'],
                analysis['Summary']
            ])

        df = pd.DataFrame(data, columns=[
            'Ticker', 'MACD', 'Signal', 'RSI', 'Upper Band', 'Lower Band', 'Implied Volatility', 'Suggestion', 'Action', 'Order Type',
            'Time in Force', 'Option Expiration', 'Option Strike Price', 'Option Premium',
            'Option Breakeven', 'Percent to Breakeven', 'Potential Profit', 'Summary'
        ])

        clear_output(wait=True)
        display(df)
        time.sleep(interval)

ticker = input("Enter the stock ticker (e.g., GME for GameStop): ")
interval = int(input("Enter the update interval in seconds: "))
track_stock(ticker, interval)


,Ticker,MACD,Signal,RSI,Upper Band,Lower Band,Implied Volatility,Suggestion,Action,Order Type,Time in Force,Option Expiration,Option Strike Price,Option Premium,Option Breakeven,Percent to Breakeven,Potential Profit,Summary
0,msft,-5.440304,-7.8982,49.494981,439.079335,390.815135,0.618339,Call (high volatility - consider risks),Buy,Limit,GTC,2024-08-23,340.0,61.0,401.0,-4.870353,20.529999,$msft 340.0 CALL (3dte) @61.00
